Create conversational agent

Combining tool usage with chat memory, create chatbot like chatgpt.

Agent Basics:

1. Combination of language models and codes
2. LLM reasons about what steps to take and call for actions.

Agent Loop:

1. Choose a tool to use.
2. Observe the output of a tool
3. Repeat until a stopping condition met.

Stopping conditions can be:

1. LLM determined :- idea of agent finish
2. Hardcoded rules/

In [ ]:
import os
from dotenv import load_dotenv


In [ ]:
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [ ]:
from langchain.tools import tool

In [ ]:
import requests
from pydantic import BaseModel, Field
import datetime

# Define the input schema

class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")
    
@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""
    
    BASE_URL = "https://api.open-meteo.com/v1/forecast?"
    
    #Parameters 
    
    params = {
        'latitude':latitude,
        'longitude':longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,       
    }
    
    # Make a request
    
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")
    
    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z','+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']
    
    closest_time_index = min(range(len(time_list)),key = lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]
    
    return f'The current temperature is {current_temperature} C'



In [ ]:
import wikipedia

@tool

def search_wikipedia(query:str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    
    for page_title in page_titles[:3]:
        try:
            wiki_page = wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\n Summary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)
        
    

In [ ]:
tools = [get_current_temperature,search_wikipedia]

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

functions = [format_tool_to_openai_function(f) for f in tools]
model = ChatOpenAI(temperature=0).bind(functions = functions)
prompt = ChatPromptTemplate.from_messages([
    ("system","You are helpful but sassy assistant"),
    ("user","{input}"),
])

chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [ ]:
result = chain.invoke({"input":"What is the weather in SF?"})

In [ ]:
result.tool

In [ ]:
result.tool_input

What we want to do here is 

create a loop that determines what tool to use, then calls the tool, and passes it back in and repeats until some stopping criteria is met. 

if we look at the prompt, what is means is if we look at the prompt, we need a place int the prompt to pass back in this of tools that are called

and corresponding outputs.

So we goona have to change the prompt little bit, specially we want to add in the place where we can pass in a list of messages.

because we want to convert this tool selection to tool observation into a list of messages and pass it back in.

to do this we gonna use messages placeholder for this listed messages and then in template, we are going to have same first two elements,

ie, system and user and then we are going to add in this agent scratchpad messages placeholder.

In [ ]:
from langchain.prompts import MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages([
    ("system","You are helpful but sassy assistant"),
    ("user","{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad") # action and observation pairs.
])

In [ ]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [ ]:
result1 = chain.invoke({
    "input":"What is the weather in SF?",
    "agent_scratchpad": [] # as we have define variable above and empty list because we haven't taken any actions yet.
})

In [ ]:
result1.tool

In [ ]:
observation = get_current_temperature(result1.tool_input)

In [ ]:
observation

In [ ]:
type(result1)

Now how to take this two things observations and type and convert it into a list and pass into agent scratchpad?

using function call format_to_openai_functions

In [ ]:
from langchain.agents.format_scratchpad import format_to_openai_functions


In [ ]:
result1.message_log

We will take message log of message1 :- ie list of messages that makes up how we arrive at this current agent action.

This contains the chat message that has additional quarks with this function call saying get current temperature and then return the 

arguments with the JSON string, that the exact response from OpenAI.

We are keeping this around in message log because, now when it comes time, to construct this agent scratch pad, we can just put it right back in there,

We are gonna use observation, which is just a string, we gonna use the function message type, we gonna pass that in

In [ ]:
format_to_openai_functions([(result1, observation), ])  # passing list of tuples corresponding to agent action and the observation.

Why list of tuples, if we add in more steps for the agent to take we can just keep on passing in a list of tuples and it will keep on generating 

this list of messages.

In [ ]:
result2 = chain.invoke({
    "input": "What is the weather in SF?",
    "agent_scratchpad": format_to_openai_functions([(result1,observation),])
})

In [ ]:
result2

Lets bundle all up into a function

In [ ]:
from langchain.schema.agent import AgentFinish

def run_agent(user_input):
    intermediate_steps = []
    while True:
        result = chain.invoke({
            "input":user_input,
            "agent_scratchpad": format_to_openai_functions(intermediate_steps)
        })
        if isinstance(result,AgentFinish):
            return result
        tool = {
            "search_wikipedia":search_wikipedia,
            "get_current_temperature":get_current_temperature,
        }[result.tool]
        observation = tool.run(result.tool_input)
        intermediate_steps.append((result,observation))
        

In [ ]:
from langchain.schema.runnable import RunnablePassthrough    # RunnablePassThrough takes initial input and passes it through.

agent_chain = RunnablePassthrough.assign(                    # Assign method, created a new argument to the dictionary that's getting passed through has a name agent_scratchpad.
    agent_scratchpad = lambda x: format_to_openai_functions(x["intermediate_steps"])
) | chain

We gonna pipe result of that which is dictionary, into the chain, we get full end-to-end chain. 

Which 

Takes in Input, in intermediate steps does the necessary pre-processing of the intermediate steps to create agent_scratchpad, then passes it to the prompt,

then to model, and then to the agent output parser.

In [ ]:
from langchain.schema.agent import AgentFinish

def run_agent(user_input):
    intermediate_steps = []
    while True:
        result = agent_chain.invoke({
            "input": user_input,
            "intermediate_steps": intermediate_steps,
        })
        if isinstance(result,AgentFinish):
            return result
        tool = {
            "search_wikipedia":search_wikipedia,
            "get_current_temperature":get_current_temperature,
        }[result.tool]
        observation = tool.run(result.tool_input)
        intermediate_steps.append((result,observation))

we are just calling agent chain and doing all the intermediate steps in agent chain.

In [ ]:
run_agent("what is the weather in SF?")

In [ ]:
run_agent("What is Langchain?")

In [ ]:
run_agent("hi!")

We used run_agent function :- It is a loop, that loops over LLM, decides what to do, and then takes next step.

But we have a class called agent_executer:- it is supup version of run_agent function, it also add fews things, 

1. Adds in better logging.
2. Adds in error handling. If output models outputs something which isn't json, we can handle it
3. Adds in error handling for tools. we can pass error to language model and ask it to correct it.

How to use agent_executer

In [ ]:
from langchain.agents import AgentExecutor
agent_executer = AgentExecutor(agent= agent_chain, tools = tools, verbose=True)

In [ ]:
agent_executer.invoke({"input":"What is langchain?"})

Conversation using agents

In [ ]:
agent_executer.invoke({"input":"My name is utkarsh"})

In [ ]:
agent_executer.invoke({"input":"What is my name?"})

Here we can see, had just told the name but it failed to remember it. what's happening?

We haven't added any mechanism to actually pass in previous messages. So it actually doesn't remember that my name is Utkarsh.

Lets add in and see whats happen after that.

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system","You are helpful but sassy assistant"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user","{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [ ]:
agent_chain = RunnablePassthrough.assign(
    agent_scratchpad = lambda x: format_to_openai_functions(x["intermediate_steps"])
) | prompt | model | OpenAIFunctionsAgentOutputParser()

create simple memory object

In [ ]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history") # return_messages = True means will return it as list of messages as we are using MessagePlaceholder. if its False will return it as string

In [ ]:
agent_executer = AgentExecutor(agent=agent_chain, tools = tools, verbose= True, memory=memory)

In [ ]:
agent_executer.invoke({"input":"My name is Utkarsh"})

In [ ]:
agent_executer.invoke({"input":"what is my name?"})

In [ ]:
agent_executer.invoke({"input":"what is weather in SF?"})

Create a Chatbot

In [ ]:
import panel as pn  
pn.extension()
import param

In [ ]:
class cbfs(param.Parameterized):
    
    def __init__(self, tools, **params):
        super(cbfs, self).__init__(**params)
        self.panels = []
        self.functions = [format_tool_to_openai_function(f) for f in tools]
        self.model = ChatOpenAI(temperature=0.0).bind(functions=self.functions)
        self.memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")
        self.prompt = ChatPromptTemplate.from_messages([
            ("system","You are helpful but sassy assistant"),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user","{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ])
        self.chain = RunnablePassthrough.assign(
            agent_scratchpad = lambda x: format_to_openai_functions(x["intermediate_steps"])
        ) | self.prompt | self.model | OpenAIFunctionsAgentOutputParser()
        self.qa = AgentExecutor(agent=self.chain, tools = tools, verbose = True, memory=self.memory)
        
    
    def convchain(self,query):
        if not query:
            return
        inp.value = ''
        result = self.qa.invoke({"input":query})
        self.answer = result['output']
        self.panels.extend([
            pn.Row("User: ", pn.pane.Markdown(query,width=450)),
            pn.Row('Chatbot: ', pn.pane.Markdown(self.answer, width=450, styles = {'background-color':'#F6F6F6'}))
        ])
        return pn.WidgetBox(*self.panels,scroll=True)
    
    def clr_history(self,count=0):
        self.chat_history = []
        return

In [ ]:
cb = cbfs(tools)

inp = pn.widgets.TextInput(placeholder="Enter text here...")
conversation = pn.bind(cb.convchain, inp)

tab1 = pn.Column(
    pn.Row(inp),
    pn.layout.Divider(),
    pn.panel(conversation, loading_indicator=True, height=400),
    pn.layout.Divider(),
)

dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('# QnA_Bot')),
    pn.Tabs(('conversation',tab1)),
)
dashboard